# **Game Sales Analysis Project**

> ### 다음 분기에 어떤 게임을 설계해야 할까


> 1.   *지역에 따라 선호하는 게임 장르가 다른지?*
2.  *연도별 게임의 트렌드가 있는지?*
3. *출고량 높은 게임에 대한 분석 및 시각화*













## **1. 데이터의 전처리**

---




> *   **라이브러리 로드, 한글폰트 설정**

















In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import stats
mpl.rc('font', family='NanumGothic')

In [ ]:
df = pd.read_csv("vgames2.csv")
df

In [ ]:
df.info()
df = df.drop(columns=['Unnamed: 0'], axis=1)
df

> *   **결측치 확인 및 제거**



> null 값은 데이터 양에 비해 미미하기 때문에 삭제


In [ ]:
df.isnull().sum()
df = df.dropna()



> *   **Sales 항목에서 단위를 M으로 통일해주고, float로 변환**







In [ ]:
#K를 M으로 변환하고 float로 변환
df['NA_Sales'] = df['NA_Sales'].replace({'[K]': '*0.001', '[M]':'*1'}, regex = True).map(pd.eval).astype(float)
df['EU_Sales'] = df['EU_Sales'].replace({'[K]': '*0.001', '[M]':'*1'}, regex = True).map(pd.eval).astype(float)
df['JP_Sales'] = df['JP_Sales'].replace({'[K]': '*0.001', '[M]':'*1'}, regex = True).map(pd.eval).astype(float)
df['Other_Sales'] = df['Other_Sales'].replace({'[K]': '*0.001', '[M]':'*1'}, regex = True).map(pd.eval).astype(float)

#Sales의 합 column 추가
df['Total_Sales'] = df['NA_Sales']+df['EU_Sales']+df['JP_Sales']+df['Other_Sales']

df.info()


> *   **year 열에서 0,1,2,3과 같은 한 자리 수 값 변환**
*2017년, 2020년 데이터 수가 적기 때문에 drop

In [ ]:
#한 자리수 년도 변환
def Year(num):
  if num <= 20:
    num = num + 2000
    return num
  elif num >20 and num <=100:
    num = num + 1900
    return num
  else:
    return num

df['Year'] = df['Year'].apply(Year).round(0).astype(int)

   
  
#2017년, 2020년 년도 drop
df_idx1 = df[df['Year']==2017].index
df_idx2 = df[df['Year']==2020].index
df = df.drop(df_idx1)
df = df.drop(df_idx2)
df

## 2. 지역에 따라 선호하는 게임 

---





> 1.   지역별 전체 게임 출고량 비교
2.   지역별 선호하는 게임 장르
3. 지역별 선호하는 게임 플랫폼






In [ ]:
#장르에 따른 지역별 판매량
df_region_genre = df.groupby(['Genre'])['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'].sum().unstack().to_frame().reset_index()
df_region_genre.rename(columns = {'level_0' : 'Region'}, inplace = True)
df_region_genre.rename(columns = {0:'Sales'}, inplace = True)
df_region_genre.head()

In [ ]:
#지역별 전체 게임 판매량
df_region = df_region_genre.drop('Genre', axis = 1)
df_region = df_region_genre.groupby(['Region'])['Sales'].sum().to_frame().reset_index()
df_region

In [ ]:
#지역별 전체 게임 판매량 시각화
sns.barplot(x='Region', y='Sales', data=df_region)
plt.title("Total game sales by region")
plt.show()

     미국에서의 게임 판매량이 가장 높다

In [ ]:
#최근 10년간 지역별 게임 판매량
df_new_filter = df.Year > 2010
df_new = df.loc[df_new_filter].reset_index(drop=True)
df_region_new = df_new.groupby(['Genre'])['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'].sum().unstack().to_frame().reset_index()
df_region_new.rename(columns = {'level_0' : 'Region'}, inplace = True)
df_region_new.rename(columns = {0:'Sales'}, inplace = True)
df_region_new.head()

In [ ]:
#최근10년 지역별 게임 판매량 시각화
sns.barplot(x='Region', y='Sales', data=df_region_new)
plt.title("2010-2016 game sales by region")
plt.show()

     최근 10년 동안에도 미국에서의 게임 판매량이 제일 많았다.

In [ ]:
#지역별 선호 장르 시각화
plt.rc('font', family='NanumGothic')
con_genre_NA = df.groupby(['Genre'])['NA_Sales'].sum().to_frame().reset_index()
con_genre_EU = df.groupby(['Genre'])['EU_Sales'].sum().to_frame().reset_index()
con_genre_JP = df.groupby(['Genre'])['JP_Sales'].sum().to_frame().reset_index()
con_genre_OT = df.groupby(['Genre'])['Other_Sales'].sum().to_frame().reset_index()

plt.figure(figsize=(8,8))
plt.pie(con_genre_NA['NA_Sales'], labels = con_genre_NA['Genre'], autopct='%.1f%%', startangle=260, counterclock=False)
plt.title('북미 지역의 선호장르')

plt.figure(figsize=(8,8))
plt.pie(con_genre_EU['EU_Sales'], labels = con_genre_EU['Genre'], autopct='%.1f%%')
plt.title('유럽 지역의 선호장르')

plt.figure(figsize=(8,8))
plt.pie(con_genre_JP['JP_Sales'], labels = con_genre_JP['Genre'], autopct='%.1f%%')
plt.title('일본 지역의 선호장르')

plt.figure(figsize=(8,8))
plt.pie(con_genre_OT['Other_Sales'], labels = con_genre_OT['Genre'], autopct='%.1f%%')
plt.title('그 외 지역의 선호장르')

      1. 북미지역 선호 장르 : Action, Sports, Shooter
      2. 유럽지역 선호 장르 : Action, Sports, Shooter
      3. 일본지역 선호 장르 : Role-Playing, Action, Sports
      4. 그 외 지역 선호 장르 : Action, Sports, Shooter

In [ ]:
con_genre_NA1 = df.groupby(['Genre'])['NA_Sales'].sum()
con_genre_EU1 = df.groupby(['Genre'])['EU_Sales'].sum()
con_genre_JP1 = df.groupby(['Genre'])['JP_Sales'].sum()
con_genre_OT1 = df.groupby(['Genre'])['Other_Sales'].sum()
con_genre = pd.DataFrame()
con_genre = con_genre.append(con_genre_NA1)
con_genre = con_genre.append(con_genre_EU1)
con_genre = con_genre.append(con_genre_JP1)
con_genre = con_genre.append(con_genre_OT1)
con_genre = con_genre.T
con_genre

     파이 차트로 판단하기보다는 신뢰성을 높이기 위해 카이제곱검정 실행
     - 귀무가설 : 지역과 선호 장르의 관계는 서로 독립이다.(연관이 없다)
     - 대립가설 : 지역과 선호 장르의 관계는 서로 연관성이 있다.



In [ ]:
# 카이 제곱검정 실행 (지역별로 선호하는 장르가 다른가?)
from scipy.stats import chi2_contingency
chi2_val, p, dof, expected=chi2_contingency(con_genre, correction=False)
if(p<0.05) :
   print('P value:', p, '\n'+'귀무가설 기각, 지역과 선호 장르의 관계는 연관이 있다')
else :
   print('P value:', p, '\n'+'귀무가설 기각못함, 지역과 선호 장르의 관계는 연관이 없다')

In [ ]:
#지역에 따른 선호 플랫폼 (상위10개)
con_plat_NA = df.groupby(['Platform'])['NA_Sales'].sum().to_frame().reset_index()
plat_NA = con_plat_NA.sort_values(by="NA_Sales", ascending=False).head(10)
con_plat_EU = df.groupby(['Platform'])['EU_Sales'].sum().to_frame().reset_index()
plat_EU = con_plat_EU.sort_values(by="EU_Sales", ascending=False).head(10)
con_plat_JP = df.groupby(['Platform'])['JP_Sales'].sum().to_frame().reset_index()
plat_JP = con_plat_JP.sort_values(by="JP_Sales", ascending=False).head(10)
con_plat_OT = df.groupby(['Platform'])['Other_Sales'].sum().to_frame().reset_index()
plat_OT = con_plat_OT.sort_values(by="Other_Sales", ascending=False).head(10)

plt.figure(figsize=(8,8))
plt.pie(plat_NA['NA_Sales'], labels = plat_NA['Platform'], autopct='%.1f%%', startangle=260, counterclock=False)
plt.title('북미 지역의 선호 플랫폼')

plt.figure(figsize=(8,8))
plt.pie(plat_EU['EU_Sales'], labels = plat_EU['Platform'], autopct='%.1f%%')
plt.title('유럽 지역의 선호 플랫폼')

plt.figure(figsize=(8,8))
plt.pie(plat_JP['JP_Sales'], labels = plat_JP['Platform'], autopct='%.1f%%')
plt.title('일본 지역의 선호 플랫폼')

plt.figure(figsize=(8,8))
plt.pie(plat_OT['Other_Sales'], labels = plat_OT['Platform'], autopct='%.1f%%')
plt.title('그 외 지역의 선호 플랫폼')

     1. 북미지역 선호 플랫폼 : X360, PS2, Wii
     2. 유럽지역 선호 플랫폼 : PS3, PS2, X360
     3. 일본지역 선호 플랫폼 : DS, PS, PS2
     4. 그 외 지역 선호 플랫폼 : PS2, PS3, X360

In [ ]:
con_plat_NA1 = df.groupby(['Platform'])['NA_Sales'].sum()
con_plat_EU1 = df.groupby(['Platform'])['EU_Sales'].sum()
con_plat_JP1 = df.groupby(['Platform'])['JP_Sales'].sum()
con_plat_OT1 = df.groupby(['Platform'])['Other_Sales'].sum()

con_pl = pd.DataFrame()
con_pl = con_pl.append(con_plat_NA1)
con_pl = con_pl.append(con_plat_EU1)
con_pl = con_pl.append(con_plat_JP1)
con_pl = con_pl.append(con_plat_OT1)
con_pl = con_pl.T
con_pl.head()

    신뢰성을 높이기 위해 카이제곱검정 실행
    - 귀무가설 : 지역별로 선호하는 플랫폼의 관계는 서로 독립이다.(지역마다 선호하는 플랫폼이 같다)
    - 대립가설 : 지역별로 선호하는 플랫폼의 관계는 서로 연관성이 있다.(지역마다 선호하는 플랫폼이 다르다.)

In [ ]:
# 카이 제곱검정 실행 (지역별로 선호하는 플랫폼이 다른가?)
from scipy.stats import chi2_contingency
chi2_val, p, dof, expected=chi2_contingency(con_pl, correction=False)
if(p<0.05) :
   print('P value:', p, '\n'+'귀무가설 기각, 지역마다 선호하는 플랫폼이 다르다')
else :
   print('P value:', p, '\n'+'귀무가설 기각못함, 지역마다 선호하는 플랫폼이 같다')

## **3. 연도별 게임의 트렌드가 있는지?**

---





> 1.   연도에 따른 지역별 판매량
2.   연도에 따른 장르별 판매량
3. 연도에 따른 장르 트렌드
   - 게임 출시 빈도수 (장르)

In [ ]:
#연도에 따른 지역별 판매량
year = df.loc[:,['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
year_sale = year.groupby('Year').sum()
year_sale.head()


In [ ]:
#연도에 따른 지역별 판매량 시각화
fig = plt.figure(figsize=(30,10))

plt.plot(year_sale.index, year_sale.NA_Sales, label='NA_Sales')
plt.plot(year_sale.index, year_sale.EU_Sales, label='EU_Sales')
plt.plot(year_sale.index, year_sale.JP_Sales, label='JP_Sales')
plt.plot(year_sale.index, year_sale.Other_Sales, label='Other_Sales')

plt.xticks(year_sale.index, rotation=45)
plt.yticks(np.arange(0,400,50))

plt.annotate('North America', xy=(2008,340), xytext=(2008,340), fontsize=15, ha='center')
plt.annotate('Europe', xy=(2008,180), xytext=(2008,180), fontsize=15, ha='center')
plt.annotate('Others', xy=(2008,85), xytext=(2008,85), fontsize=15, ha='center')
plt.annotate('Japan', xy=(2008,52), xytext=(2008,52), fontsize=15, ha='center')

plt.title('연도에 따른 지역별 출고량', fontsize=22)
plt.ylabel('Sales(단위 : M)', fontsize=15)
plt.xlabel('연도', fontsize=15)
plt.legend()
plt.grid(True, color='gray', alpha=0.5, linestyle='--')
plt.show()

     2010년경에 전세계적으로 게임 판매량이 증가했고 북미, 유럽, 일본 순으로 게임이 많이 판매되었다.
     현재까지 전체 게임 판매량의 순위도 북미, 유럽, 일본 순으로 유지중

In [ ]:
#연도에 따른 장르별 판매량
year_sale_gr = df.groupby(by=['Year', 'Genre'])['Total_Sales'].sum().unstack()
year_sale_gr = year_sale_gr.fillna(0)
year_sale_gr = year_sale_gr.reset_index()
year_sale_gr.head()

In [ ]:
#연도에 따른 장르별 판매량 시각화
years1 = year_sale_gr['Year']
plt.figure(figsize = (30,15))


plt.plot(years1, year_sale_gr['Action'], label = 'Action')
plt.plot(years1, year_sale_gr['Adventure'], label = 'Adventure')
plt.plot(years1, year_sale_gr['Fighting'], label = 'Fighting')
plt.plot(years1, year_sale_gr['Misc'], label = 'Misc')
plt.plot(years1, year_sale_gr['Platform'], label = 'Platform')
plt.plot(years1, year_sale_gr['Puzzle'], label = 'Puzzle')
plt.plot(years1, year_sale_gr['Racing'], label = 'Racing')
plt.plot(years1, year_sale_gr['Role-Playing'], label = 'Role-Playing')
plt.plot(years1, year_sale_gr['Shooter'], label = 'Shooter')
plt.plot(years1, year_sale_gr['Simulation'], label = 'Simulation')
plt.plot(years1, year_sale_gr['Sports'], label = 'Sports')
plt.plot(years1, year_sale_gr['Strategy'], label = 'Strategy')

plt.xticks(years1, rotation=45)
plt.yticks(np.arange(0,150,5))

plt.legend()

plt.title('연도별 장르에 따른 전 지역 판매량', fontsize=22)
plt.ylabel('Sales(단위 : M)', fontsize=15)
plt.xlabel('연도', fontsize=15)
plt.legend()
plt.grid(True, color='gray', alpha=0.5, linestyle='--')

plt.show()


     Action, Sports 게임이 이전부터 인기있었으며 현재까지도 인기있는 장르입니다.



      위 그래프로는 정확하게 판단이 안되기 때문에 연도별 가장 인기 있는 장르도 분석 



In [ ]:
#연도별 가장 인기있는 게임 트랜드
year_sale2 =df.groupby(by=['Year', 'Genre'])['Total_Sales'].sum().reset_index()
year_sale_max = year_sale2.loc[year_sale2.groupby(['Year'])['Total_Sales'].idxmax()]
year_sale_max = year_sale_max.reset_index().drop('index',axis=1)
year_sale_max.head()

In [ ]:
#연도별 가장 인기있는 게임 트랜드 시각화
genre = year_sale_max['Genre']

plt.figure(figsize=(30,20))
bar = sns.barplot(x='Year', y = 'Total_Sales', data=year_sale_max)
index=0
for v in year_sale_max['Total_Sales']:
    bar.text(index, v+1, str(genre[index] + '-' +str(round(v, 1))), color='black', size=20, rotation=90, ha="center")
    index +=1

plt.title("연도별 가장 인기있는 게임 트렌드", fontsize=30)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Total Sales", fontsize=20)

plt.xticks(rotation=90)
plt.grid(True)
plt.show()

     2000년대부터 스포츠게임, 액션게임의 인기가 올라갔다.
     2006년, 2009년 제외하고는 주로 스포츠게임보다는 액션게임이 더 인기가 있다.




> *   시대별 수요 트렌드 그래프 (10년 주기)





In [ ]:
df_per = df.groupby(by=['Year', 'Genre'])['Total_Sales'].sum().unstack()
df_per = df_per.fillna(0)
df_per.loc['1980-1989', :] = df_per.loc[[1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989]].sum()
df_per.loc['1990-1999', :] = df_per.loc[[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]].sum()
df_per.loc['2000-2009', :] = df_per.loc[[2000,2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]].sum()
df_per.loc['2010-2016', :] = df_per.loc[[2010, 2011, 2012, 2013, 2014, 2015, 2016]].sum()
df_per = df_per.loc[['1980-1989', '1990-1999', '2000-2009', '2010-2016']]
df_per.head()

In [ ]:
plt.figure(figsize=(20,10))
plt.title('시대별 장르 트렌드', fontsize=20)
plt.ylabel('판매량(단위:M)')
plt.grid(True)
sns.lineplot(data=df_per)


     2000년대 초가 게임 시장의 정점
     10년 주기로 봤을 때도 2000년도 이후로부터는 액션 장르가 압도적으로 인기가 있다
     2010년 이후로는 Sports 장르가 3위로 밀려남

### 위 결과를 바탕으로 Action과 Sports 장르가 인기 있다고 판단
### 게임 회사들도 위 장르의 게임을 많이 판매했을까?

In [ ]:
#장르별 게임 출시 횟수 분석
df_gen_count = df.groupby(['Genre']).count()
df_gen_count = df_gen_count['Name'].to_frame()
df_gen_count.rename(columns={'Name':'Total_count'}, inplace = True)
df_gen_count.head()

In [ ]:
#장르별 출시 빈도 시각화

df_gen_count.sort_values(by=['Total_count']).plot(kind='barh', title='장르별 출시 빈도')

     Action, Sports 장르 게임 수요가 많으니 공급도 많음.

## **4. 최근 10년간 판매량이 높은 게임에 대한 분석**
### 새로 출시할 게임에 대한 주제이므로 최근 데이터를 중심으로 분석
---





> 1.   판매량이 가장 많은 30개의 게임 (플랫폼별)
2.   판매량이 가장 많은 30개의 게임 (장르별)
3. 판매량이 가장 많은 3개의 장르 분석

In [ ]:
#판매량이 가장 많은 30개의 게임(플랫폼별)
sales_top30 = df[df.Year > 2010].sort_values(by='Total_Sales', ascending = False)
sales_top30 = sales_top30.head(30).reset_index(drop=True)
sales_top30.head()

In [ ]:
#최근 10년 판매량이 가장 많은 30개의 게임(플랫폼별) 시각화
top30_plat = sales_top30.groupby(['Platform'])['Total_Sales'].sum().reset_index()
platform = top30_plat['Platform'].values
plt.figure(figsize=(20,10))
top30_plat_sales = sns.barplot(x='Total_Sales', y='Platform', data=top30_plat)
index = 0
for value in top30_plat['Total_Sales']:
    top30_plat_sales.text(x=value +1, y=index, s=str(round(value,2)), color='black', size=10)
    index+=1

plt.show()


    최근 10년간 출고량이 많았던 플랫폼은 X360, PS3, PS4, 3DS 순으로 많았다.

In [ ]:
#판매량이 가장 많은 30개의 게임 (장르별)
top30_gen = sales_top30.groupby(['Genre'])['Total_Sales'].sum().reset_index()
genre = top30_gen['Genre'].values
plt.figure(figsize=(20,10))
top30_gen_sales = sns.barplot(x='Total_Sales', y='Genre', data=top30_gen)
index = 0

for value in top30_gen['Total_Sales']:
    top30_gen_sales.text(x=value+1, y=index, s=str(round(value,2)), color='black', size=10)
    index+=1
    
plt.show()

     최근 10년간 판매량이 많은 장르는 Shooter, Action 순으로 많았다.

## **5. 게임 장르와 플랫폼과의 연관성**

---





> 1.   가설 설정 및 확인

       
 > 2.  어떤 장르에서 어떤 플랫폼이 판매량이 많은가


     게임 장르와 플랫폼은 서로 독립이다(연관이 없다) - 귀무 가설
       
     게임 장르와 플랫폼은 서로 연관이 있다 - 대립 가설

In [ ]:
df_crosstable = pd.crosstab(df['Genre'], df['Platform'])
df_crosstable.head()

In [ ]:
# 카이 제곱검정 실행 (게임 장르와 플랫폼은 서로 독립(연관이 없다))일까?
from scipy.stats import chi2_contingency
chi2_val, p, dof, expected=chi2_contingency(df_crosstable, correction=False)
if(p<0.05) :
   print('P value:', p, '\n'+'귀무가설 기각, 게임 장르와 플랫폼은 연관이 있다')
else :
   print('P value:', p, '\n'+'귀무가설 기각못함, 게임 장르와 플랫폼은 연관이 없다')

     최근 10년간 데이터를 바탕으로, 장르에 따른 각 플랫폼별 게임 판매량 시각화

In [ ]:
#최근 10년간 장르에 따른 각 플랫폼별 게임 판매량
df_10 = df[df.Year > 2010]
df_genre_plat = df_10.groupby(['Genre', 'Platform'])['Total_Sales'].sum().reset_index()
df_genre_plat.head()


In [ ]:
#최근 10년간 장르에 따른 각 플랫폼별 게임 판매량 시각화
plt.figure(figsize=(15,15))
plt.scatter(df_genre_plat['Genre'], df_genre_plat['Platform'], s=df_genre_plat['Total_Sales']*50, c=df_genre_plat['Total_Sales'], cmap='viridis', alpha=0.4)
plt.title('2010-2016 장르에 따른 각 플랫폼별 판매량')
plt.xlabel('Genre')
plt.ylabel('Platform')
plt.colorbar()
plt.grid(True)
plt.show()

    1위: Action 장르에서 PS3 플랫폼
    2위 : Shooter 장르에서 X360 플랫폼
    3위 : Action 장르에서 X360 플랫폼
    4위 : Shooter 장르에서 PS3 플랫폼



---
 - 위 그래프에서 인기 있는 플랫폼은 PS3, X360, PS4, XOne인데,

 - 게임 플랫폼의 신규 업데이트, 단종 문제 등으로 ***최근 10년간의 분석보다는 전체 연도별 게임 플랫폼 트렌드를 분석 진행***

In [ ]:
#연도별 플랫폼 출시 비율 꺾은선그래프
sns.set_palette('deep')

plt.figure(figsize=(10,7))
plt.subplots_adjust(wspace=0.2, hspace=0.5)

PS3_filter = df.Platform =='PS3'
df_PS3 = df.loc[PS3_filter]
df_PS3 = df_PS3.groupby(['Year', 'Platform']).size().reset_index(name = 'Count')
plt.plot(df_PS3['Year'], df_PS3['Count'],marker='o', label = 'PS3')

PS4_filter = df.Platform =='PS4'
df_PS4 = df.loc[PS4_filter]
df_PS4 = df_PS4.groupby(['Year', 'Platform']).size().reset_index(name = 'Count')
plt.plot(df_PS4['Year'], df_PS4['Count'],marker='o', label = 'PS4')

X360_filter = df.Platform =='X360'
df_X360 = df.loc[X360_filter]
df_X360 = df_X360.groupby(['Year', 'Platform']).size().reset_index(name = 'Count')
plt.plot(df_X360['Year'], df_X360['Count'],marker='o', label = 'X360')

XOne_filter = df.Platform =='XOne'
df_XOne = df.loc[XOne_filter]
df_XOne = df_XOne.groupby(['Year', 'Platform']).size().reset_index(name = 'Count')
plt.plot(df_XOne['Year'], df_XOne['Count'],marker='o', label = 'XOne')

plt.legend()
plt.title('각 플랫폼의 출시량')
plt.xlabel('연도')
plt.ylabel('출시량')
plt.show()

### - PS3, X360은 PS4, XOne이 생겨서 출시량이 감소하는 추세
### - PS3의 후속인 PS4 를 선택하여 분석 진행할 것이다

## **6. PS4플랫폼을 기반으로 한 Action 장르 분석**

---





> 1.  어느 지역에서 가장 큰 매출을 달성했는가?

       
> 2.  제일 많이 판매된 회사의 매출

     PS4 플랫폼을 기반으로 한 Action 게임이 최근 어느지역에서 가장 큰 매출을 달성했는가?

In [ ]:
#PS4 플랫폼을 기반으로 한 Action 게임 분석
Genre_filt = (df.Genre == 'Action') & (df.Year > 2010)
df_action = df.loc[Genre_filt].reset_index(drop=True)

plat_filt = (df_action.Platform =='PS4')
df_action_ps4 = df_action.loc[plat_filt].reset_index(drop=True)
df_action_ps4

In [ ]:
 #PS4 플랫폼을 기반으로 한 Action 게임이 최근 어느지역에서 가장 큰 매출을 달성했는가?
plt.figure(figsize=(7,10))
area_NA = df_action_ps4['NA_Sales'].sum()
area_EU = df_action_ps4['EU_Sales'].sum()
area_JP = df_action_ps4['JP_Sales'].sum()
area_OT = df_action_ps4['Other_Sales'].sum()
area = pd.DataFrame({'area':['NA', 'EU', 'JP', 'Others'],'Sales':[area_NA, area_EU, area_JP, area_OT]})
area_barplot = sns.barplot(x='area', y='Sales', data=area)
Sales = area['Sales'].values

index=0
for value in area['Sales']:
    area_barplot.text(x=index, y=value+0.3, s=str(str(round(value,2))), color='black', size=12, ha='center')
    index+=1

plt.title('PS4 플랫폼 기반 Action 게임 지역별 매출량 (2010-2016)', size=15)
plt.xlabel('지역')
plt.ylabel('Sales')
plt.show()

     유럽 지역에서 PS4 플랫폼을 기반으로 한 Action장르 게임 판매량이 제일 높았다.

In [ ]:
#유럽 내 PS4,Action게임 출고 회사중에서 제일 많이 판매된 회사 (최근10년)
df_action_ps4_pu = df_action_ps4.groupby(['Publisher'])['EU_Sales'].sum().reset_index().sort_values(by='EU_Sales', ascending=False)
plt.figure(figsize=(10,10))
pu_barplot = sns.barplot(x='EU_Sales', y='Publisher', data=df_action_ps4_pu)
plt.title('EU 지역의 PS4플랫폼 기반 Action게임 회사별 매출량 (2010-2016)')
plt.xlabel('EU지역 매출량')
plt.ylabel('회사')
plt.grid(True)
plt.show()


     Ubisoft의 매출량이 가장 높기 때문에 Ubisoft사의 게임을 벤치마킹해 설계한다.

## 분석 결과 : 


1. 전체 기간, 최근 10년동안 북미에서의 게임 판매량이 가장 많다.
   - 최근 10년동안은 유럽과 북미의 판매량 차이 미미
2. 일본 제외한 나머지 지역 선호 장르 : Action, Sports
3. 북미지역 선호 플랫폼 : X360, PS2, Wii
4. 유럽지역 선호 플랫폼 : PS3, PS2, X360
3. 2000년도 이후부터는 Action 장르가 압도적으로 인기가 있다.
4. 최근 10년간 출고량이 많았던 플랫폼은 X360, PS3, PS4, 3DS 순으로 많았다.
5. 최근 10년간 PS3플랫폼을 기반으로 한 Action 장르 게임이 제일 많이 판매되었다.
  - 전체 기간동안 출고량과 비교하여 신규 플랫폼인 PS4 채택
6. 유럽 지역에서 PS4 플랫폼을 기반으로 한 Action장르 게임 판매량이 제일 높았다. 
7. Ubisoft의 매출량이 가장 높음, Ubisoft사의 게임을 벤치마킹해 설계한다.



---

## 결론 : PS4 플랫폼을 기반으로 한 Action 장르 게임을 Ubisoft 사를 벤치마킹해 유럽을 타겟으로 판매해야 한다.

 
    
